In [1]:
# Last Dance

In [226]:
# Define Imports and Paths
import re
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from pydicom.dataset import Dataset


path_to_pacs = Path('/Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/pacs.json.csv')
path_to_ris = Path('/Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/ris.csv')
path_to_lim = Path('/Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/lims.txt')



In [273]:
# 1 Step. Retrieve and form PAC full dataset

# PAC data <-- all the imaging data from the PACS, one line per image
# - patient_uid - non unique
# - patient_access_number_id - non unique 
# but if drop duplicates by "patient_access_number_id",
# then get desired number == 248 rows (UNIQUE*) 

# Read as PAC file
with open(path_to_pacs, 'r') as f:
    pac_data = f.readlines() # List[nested dicts],
                             # where each item of list (904 items)
                             # is dedicated examination 

rows = []

for patient_index, patient in enumerate(tqdm(pac_data)):
    buf_dicom = Dataset.from_json(patient)
    
    if patient_index == 9:
        wrong_df = Dataset.from_json(patient)
        
    # Patient ID 
    patient_id = buf_dicom.PatientID # can be wrong, need check with assert later

    # Patient Gender 
    patient_gender = buf_dicom.PatientSex
    
    # Date of birth
    patient_date_of_birth = buf_dicom.PatientBirthDate
    patient_date_of_birth = datetime.strptime(str(patient_date_of_birth), '%Y%M%d')
    # Patient date of birth is same for all PATIENTS(BUG IN DATA) 
    ## IT MEANS BETTER TO GREP DATE OF BIRTH FROM "RIS" FILE 
    patient_date_of_birth = f'{patient_date_of_birth.year}.{patient_date_of_birth.month}.{patient_date_of_birth.day}'
    
    # Access number
    patient_access_number = buf_dicom.AccessionNumber
    
    # PAC Studies {DICOM STUDIES}
    StudyID = buf_dicom.StudyID 
    StudyDate = buf_dicom.StudyDate
    StudyTime = buf_dicom.StudyTime
    StudyDescription = buf_dicom.StudyDescription
    StudyInstanceUID = buf_dicom.StudyInstanceUID
    StudyStatusID = buf_dicom.StudyStatusID
    StudyComments = buf_dicom.StudyComments
    
    buffer_row = {
        'patient_uid': patient_id,
        'sex': patient_gender,
        'date_of_birth': patient_date_of_birth, # should be obtained from RIS file
        'patient_access_number_id': patient_access_number,
        'studies': {'StudyID': salty_hash(str(StudyID)),
                    'StudyDate': StudyDate if not len(StudyDate) == 0 else np.nan,
                    'StudyTime': StudyTime if not len(StudyTime) == 0 else np.nan,
                    'StudyDescription': StudyDescription if not len(StudyDescription) == 0 else np.nan,
                    'StudyInstanceUID': StudyInstanceUID if not len(StudyInstanceUID) == 0 else np.nan,
                    'StudyStatusID': salty_hash(str(StudyStatusID)),
                    'StudyComments': StudyComments if not len(StudyComments) == 0 else np.nan,
                    }
    }
    
    rows.append(buffer_row)
    
# pass dict with all records into pandas DataFrame    
pac_df = pd.DataFrame.from_dict(rows)

# check whether we have any unique identifier of the table (column related to id)
print((pac_df['patient_uid'].is_unique))
print((pac_df['patient_access_number_id'].is_unique))

# [False, False] <- that means we currently can not use any of 2 related to id 
# columns as unique identifer of table
# As well, it we should apply some changes. For that we need RIS DataFrame.

pac_df

  0%|          | 0/904 [00:00<?, ?it/s]/Users/macbook/miniconda3/envs/ML/lib/python3.6/site-packages/pydicom/jsonrep.py:130: UserWarning: no bulk data URI handler provided for retrieval of value of data element "7FE00010"
  'of value of data element "{}"'.format(self.tag)
100%|██████████| 904/904 [00:01<00:00, 505.46it/s]

False
False


,patient_uid,sex,date_of_birth,patient_access_number_id,studies
0,1.2.826.0.1.10184103.5.1.80750,F,1888.1.1,41e1dea8-0c5d-4ca1-9fcd-041c67b2d1c0,{'StudyID': 'e4cec40844631c926e2905f514b53643c...
1,1.2.826.0.1.10184103.5.1.80750,F,1888.1.1,41e1dea8-0c5d-4ca1-9fcd-041c67b2d1c0,{'StudyID': 'e4cec40844631c926e2905f514b53643c...
2,1.2.826.0.1.10184103.5.1.80750,F,1888.1.1,41e1dea8-0c5d-4ca1-9fcd-041c67b2d1c0,{'StudyID': 'e4cec40844631c926e2905f514b53643c...
3,1.2.826.0.1.10184103.5.1.80750,F,1888.1.1,41e1dea8-0c5d-4ca1-9fcd-041c67b2d1c0,{'StudyID': 'e4cec40844631c926e2905f514b53643c...
4,1.2.826.0.1.10184103.5.1.148327,F,1863.1.28,7134ddfd-f53b-4ab1-a2c7-2b0b56c574d2,{'StudyID': 'e4cec40844631c926e2905f514b53643c...
...,...,...,...,...,...
899,1.2.826.0.1.10184103.5.1.34358,F,1898.1.31,f44ed697-4fe3-4573-b967-c66f61171d5a,{'StudyID': 'e4cec40844631c926e2905f514b53643c...
900,f5c8540f-c885-40ad-9081-1763e45fe828,F,1869.1.20,5cd192a0-fcb0-4fb8-8f29-a5f88afa7256,{'StudyID': 'e4cec40844631c926e2905f514b53643c...
901,1.2.826.0.1.10184103.5.1.93121,F,1869.1.20,5cd192a0-fcb0-4fb8-8f29-a5f88afa7256,{'StudyID': 'e4cec40844631c926e2905f514b53643c...
902,f5c8540f-c885-40ad-9081-1763e45fe828,F,1869.1.20,5cd192a0-fcb0-4fb8-8f29-a5f88afa7256,{'StudyID': 'e4cec40844631c926e2905f514b53643c...


In [236]:
# 2 Step. Retrieve and form RIS full dataset

# RIS data <-- radiologist opinions on a case
# - pat_id - non unique
# - id - UNIQUE* ('id' maps with 'accession_number_id' from PAC data)

# RIS id <--> PAC accession_number_id

# read RIS data
ris_df = pd.read_csv(path_to_ris)
ris_df['rad'] = np.nan * len(ris_df) 

# check whether we have any unique identifier of the table (column related to id)
print((ris_df['pat_id'].is_unique)) # False
print((ris_df['id'].is_unique)) # True
# Good news - ris_df['id'] column represents unique identification of table
# Plus, ris_df['id'] column is Primary Key for PAC table
# That means we can grep ris_df['id'] values from RIS and iterate trough PAC DataFrame by that values


# Transform RIS
columns = ['outcome_l', 'outcome_r']
rows = []

for row_index, row in enumerate(ris_df.itertuples()):
    voice_1 = row.outcome_l
    voice_2 = row.outcome_r
    
    sides = (row.outcome_l, row.outcome_r)
    date = datetime.strptime(str(row.date), '%Y%M%d')
    
    buff_row = {
        'side': sides,
        'date': f'{date.year}.{date.month}.{date.day}',
        'opinion': voice_1 if voice_1 == voice_2 else (voice_1, voice_2)
        }
    
    rows.append(buff_row)
    # ris_df['rad'][row_index] = buff_dict # can be replaced by iloc/loc/ix/at/iat
    # ris_df.at[row_index,'rad'] = buff_dict

ris_df['rad'] = rows
        
# # write results to file             
# with open("/Users/macbook/Desktop/full_ris_json.txt", "w") as file:
#     file.write(json.dumps(rows))


ris_df

False
True


,pat_id,pat_sex,pat_dob,id,outcome_l,outcome_r,date,rad
0,bf98f2e9-f02e-48b0-8472-470af8aebf7a,F,19620722,41e1dea8-0c5d-4ca1-9fcd-041c67b2d1c0,R2,R2,20140808,"{'side': ('R2', 'R2'), 'date': '2014.1.8', 'op..."
1,bf98f2e9-f02e-48b0-8472-470af8aebf7a,F,19620722,7134ddfd-f53b-4ab1-a2c7-2b0b56c574d2,R1,R3,20161214,"{'side': ('R1', 'R3'), 'date': '2016.1.14', 'o..."
2,bf98f2e9-f02e-48b0-8472-470af8aebf7a,F,19620722,71ad015d-b14c-45d1-8508-ea9dca9b0a9a,R1,R3,20180521,"{'side': ('R1', 'R3'), 'date': '2018.1.21', 'o..."
3,bf98f2e9-f02e-48b0-8472-470af8aebf7a,F,19620722,0fc40be6-9adc-407e-b762-0cc3341360b7,R1,R2,20200725,"{'side': ('R1', 'R2'), 'date': '2020.1.25', 'o..."
4,bf98f2e9-f02e-48b0-8472-470af8aebf7a,F,19620722,47821fb9-6aa6-44c2-9c8d-6adb6ea24251,R2,R2,20221030,"{'side': ('R2', 'R2'), 'date': '2022.1.30', 'o..."
...,...,...,...,...,...,...,...,...
243,55e8e3d4-a3be-4a91-a880-a4497e130c94,F,19700722,fe76e9da-435d-4327-945d-3edcb442cbf8,R2,R1,20220103,"{'side': ('R2', 'R1'), 'date': '2022.1.3', 'op..."
244,637f4e3c-b786-4229-a8f3-b2bd91e97265,F,19620722,0832e6de-2fc7-4655-9d58-1bff0e18b23d,R2,R3,20120824,"{'side': ('R2', 'R3'), 'date': '2012.1.24', 'o..."
245,f5c8540f-c885-40ad-9081-1763e45fe828,F,19680722,23f32605-7bf6-4505-bd31-f45d54f4917f,R1,R3,20210816,"{'side': ('R1', 'R3'), 'date': '2021.1.16', 'o..."
246,f5c8540f-c885-40ad-9081-1763e45fe828,F,19680722,f44ed697-4fe3-4573-b967-c66f61171d5a,R2,R1,20230711,"{'side': ('R2', 'R1'), 'date': '2023.1.11', 'o..."


In [279]:
# 3 Step. Retrieve and form LIMS full dataset

# LIMS data <-- pathology data from LIMS in HL7 format
# - id - UNIQUE
# LIMS id <--> RIS id

### Function wich handle LIMS file
### Returns list of external PAT_IDS + write down to file parsered data. 


count = 0
separator = '-'
rows = []

for line in open(path_to_lim):
    
    cleaned_line = re.sub('[\W_]+', ' ', line).split()
    if len(cleaned_line) == 0:
        print('---Blank line')
        continue
    
    assert cleaned_line[0] in ['MSH', 'EVN', 'PID', 'OBR', 'OBX'], 'Wrong key detected.'
    
    # work with MSH key
    if cleaned_line[0] == 'MSH':
        print('PROCESSED MSH')
        assert len(cleaned_line) == 13, 'Wrong len of MSH record.'  
        msh = separator.join(cleaned_line[1:])

        
    # work with ENV key    
    if cleaned_line[0] == 'EVN':
        print('PROCESSED EVN')
        assert len(cleaned_line) == 7, 'Wrong len of EVN record.'  
        evn = separator.join(cleaned_line[1:])
    
    
    # work with PID {Personal Patient Information} key
    if cleaned_line[0] == 'PID':
        print('PROCESSED PID')
        assert len(cleaned_line) == 8, 'Wrong len of PID record.'
          
        lims_patient_external_id = (separator.join(cleaned_line[1:6]))[3:]
        lims_patient_internal_id = separator.join(cleaned_line[1:6])
        
        
#         print(lims_patient_external_id)
#         print(lims_patient_internal_id)


    if cleaned_line[0] == 'OBR': # work with OBR key
        print('PROCESSED OBR')
        print(separator.join(cleaned_line[1:6]))
        if len(cleaned_line) != 10:
            print('Found OBR record with missing observation_id.')
            # ID
            lims_observation_request_id = np.nan
        
            # requested_date
            lims_requested_date = cleaned_line[1]
            lims_requested_date = datetime.strptime(lims_requested_date, '%Y%M%d')
            lims_requested_date = f'{lims_requested_date.year}.{lims_requested_date.month}.{lims_requested_date.day}'
        
            # observation_date
            lims_observation_date = cleaned_line[2]
            lims_observation_date = datetime.strptime(lims_observation_date, '%Y%M%d')
            lims_observation_date = f'{lims_observation_date.year}.{lims_observation_date.month}.{lims_observation_date.day}'
        
            # observation_status
            lims_observation_status = cleaned_line[3]
            
        else:
            # ID
            lims_observation_request_id = np.nan
        
            # requested_date
            lims_requested_date = cleaned_line[6]
            lims_requested_date = datetime.strptime(lims_requested_date, '%Y%M%d')
            lims_requested_date = f'{lims_requested_date.year}.{lims_requested_date.month}.{lims_requested_date.day}'
        
            # observation_date
            lims_observation_date = cleaned_line[7]
            lims_observation_date = datetime.strptime(lims_observation_date, '%Y%M%d')
            
            lims_observation_date = f'{lims_observation_date.year}.{lims_observation_date.month}.{lims_observation_date.day}'
        
            # observation_status
            lims_observation_status = cleaned_line[8]        
        
    if cleaned_line[0] == 'OBX': # work with OBX key
        print('PROCESSED OBX')
        assert len(cleaned_line) == 6, 'Wrong len of OBR record.'
        
        lims_observation_result_status = str(cleaned_line[5][0])
        lims_observation_probability = int(cleaned_line[5][1:]) / 100
        lims_result_status_repr = 'Record coming over is a correction and thus replaces a final result' 
        # if C --> Record coming over is a correction and thus replaces a final result
        # if D --> Deletes the OBX record
        # if F --> Final results; Can only be changed with a corrected result.
        # if I --> Specimen in lab; results pending
        # if P --> Preliminary results
        # if R --> Results entered -- not verified
        # if S --> Partial results
        # if U --> Results status change to final without retransmitting results already sent as ‘preliminary.’  E.g., radiology changes status from preliminary to final
        # if W --> Post original as wrong, e.g., transmitted for wrong patient
        # if X --> Results cannot be obtained for this observation

    count += 1
    
    if count == 5:  
        print('Adding to buff dict to main dict.')
        buffer_rows = {
            'id': lims_patient_external_id,
            'patho': {
                'observation_date': lims_observation_date,
                'observation_result_status': lims_observation_result_status,
                'observation_result_status_representation': lims_result_status_repr if lims_observation_result_status == 'C' else 'other',
                'MSH': msh,
                'EVN': evn,
                'PID': [{
                    'external_pat_id': lims_patient_external_id,
                    'internal_pat_id': lims_patient_internal_id
                    }],

                'OBR': [{
                    'observation_request_id': lims_observation_request_id,
                    'observation_requested_date': lims_requested_date,
                    'observation_date': lims_observation_date,
                    'observation_status': lims_observation_status
                }],
            
                'OBX': [{
                    'observation_date': lims_observation_date,
                    'observation_probability': lims_observation_probability,
                    'observation_result_status': lims_observation_result_status,
                    'observation_result_status_representation': lims_result_status_repr if lims_observation_result_status == 'C' else 'other'
                }]
            } 
        }
        rows.append(buffer_rows)
        count = 0

# # write results to file             
# with open("/Users/macbook/Desktop/lims_json.txt", "w") as file:
#     file.write(json.dumps(rows))        
        
lims_df = pd.DataFrame.from_dict(rows)
    
lims_df

PROCESSED MSH
PROCESSED EVN
PROCESSED PID
PROCESSED OBR
ef0189ad-b412-4db9-a4d2-152ac3e2ad94
PROCESSED OBX
Adding to buff dict to main dict.
---Blank line
PROCESSED MSH
PROCESSED EVN
PROCESSED PID
PROCESSED OBR
3ea1898a-046a-434a-bb0c-772b70b445a9
PROCESSED OBX
Adding to buff dict to main dict.
---Blank line
PROCESSED MSH
PROCESSED EVN
PROCESSED PID
PROCESSED OBR
1d53a30a-79b7-4949-aee7-54666a8fd820
PROCESSED OBX
Adding to buff dict to main dict.
---Blank line
PROCESSED MSH
PROCESSED EVN
PROCESSED PID
PROCESSED OBR
20310104-20310104-F-idXXX
Found OBR record with missing observation_id.
PROCESSED OBX
Adding to buff dict to main dict.
---Blank line
PROCESSED MSH
PROCESSED EVN
PROCESSED PID
PROCESSED OBR
e27e13cd-3c1e-4816-b810-d11a6103224d
PROCESSED OBX
Adding to buff dict to main dict.
---Blank line
PROCESSED MSH
PROCESSED EVN
PROCESSED PID
PROCESSED OBR
2506a2d2-851e-4bfb-85fc-5129cd27441a
PROCESSED OBX
Adding to buff dict to main dict.
---Blank line


,id,patho
0,6bb3b35a-749a-4aa0-998a-77145b176cf8,"{'observation_date': '2019.1.22', 'observation..."
1,9dd13293-ce18-4f1e-8d25-64dafb2bfd7f,"{'observation_date': '2022.1.24', 'observation..."
2,a45931c9-b895-4b49-bf25-a7ac5cc81754,"{'observation_date': '2022.1.25', 'observation..."
3,54e4035a-b9eb-4118-ba1e-635e3f934c8a,"{'observation_date': '2031.1.4', 'observation_..."
4,b4dcdc27-b876-4ee1-a24d-3d14472f96ee,"{'observation_date': '2016.1.4', 'observation_..."
5,df70ee3f-8f64-4ab4-af78-a3a739a0a347,"{'observation_date': '2023.1.19', 'observation..."


In [89]:
# ris_df[ris_df['pat_id'] == 'df70ee3f-8f64-4ab4-af78-a3a739a0a347']

,pat_id,pat_sex,pat_dob,id,outcome_l,outcome_r,date
195,df70ee3f-8f64-4ab4-af78-a3a739a0a347,F,19690722,568f7702-9f8a-4721-a347-f9df74c885f1,R1,R1,20151205
196,df70ee3f-8f64-4ab4-af78-a3a739a0a347,F,19690722,2506a2d2-851e-4bfb-85fc-5129cd27441a,R2,R5,20180615
197,df70ee3f-8f64-4ab4-af78-a3a739a0a347,F,19690722,934ba3af-d769-438e-a7a0-555e95b92046,R2,R3,20200607
198,df70ee3f-8f64-4ab4-af78-a3a739a0a347,F,19690722,82e305eb-9373-46bd-8e85-e1a25269a7b9,R1,R2,20230224


In [274]:
# 4 Step Hash IDS
# - hash patient_uid
# - hash patient_uid


SALT = 'kheiron_test_task'

def salty_hash(string: str) -> str:
    """Hash input string.
    Args:
        string (str): any string value
    Returns:
        hash of salted string
    """
    hasher = sha1()
    hasher.update(str.encode(string + SALT))
    return hasher.hexdigest()

    

def process_treatment(ris_df, pac_df, lims_df):
    rows = []
    
    for treatment_index, treatment_id in enumerate(tqdm(ris_df['id'])):
        
        patient_uid = pac_df[pac_df['patient_access_number_id'] == treatment_id]\
                      ['patient_uid'].values[0] # zeroed item, cause usually get array of equal values
        
        gender = pac_df[pac_df['patient_access_number_id'] == treatment_id]\
                 ['sex'].values[0]
        
        date_of_birth = ris_df[ris_df['id'] == treatment_id]\
                        ['pat_dob'].values[0] # 19690722
        date_of_birth = datetime.strptime(str(date_of_birth), '%Y%M%d') # 1969.07.22
        
        dicom_studies = list(pac_df[pac_df['patient_access_number_id'] == treatment_id]\
                  ['studies'].values) # LIST[DictS] 
        
        radiology_studies = ris_df['rad'].iloc[treatment_index]
             
        pathology = lims_df[lims_df['id'] == treatment_id]
        
        if pathology.shape[0] == 0:
            pathology_studies = {
                'date': np.nan,
                'opinion': np.nan
            }
            
        else:
            pathology_studies = pathology['patho'].values # List[Dicts]
            
        buff_row = {
            'patient_uid': salty_hash(patient_uid),
            'sex': gender,
            'date_of_birth': f'{date_of_birth.year}.{date_of_birth.month}.{date_of_birth.day}',
            'studies': dicom_studies,
            'rad': radiology_studies,
            'patho': pathology_studies
        }
        
        rows.append(buff_row)
        
    with open("/Users/macbook/Desktop/result_json.txt", "w") as file:
        file.write(json.dumps(rows))
        
    result_df = pd.DataFrame.from_dict(rows)
        
    return result_df        

result_df = process_treatment(ris_df, pac_df, lims_df)

100%|██████████| 248/248 [00:00<00:00, 351.74it/s]


In [276]:
# for studies in result_df['studies']:
#     for study in studies:
#         print(study['StudyID'])

result_df['studies'][0]

[{'StudyID': 'e4cec40844631c926e2905f514b53643c30cfc9a',
  'StudyDate': '20221030',
  'StudyTime': '000000.000000',
  'StudyDescription': nan,
  'StudyInstanceUID': '1.2.826.0.1.10184103.5.4.175302',
  'StudyStatusID': 'f1ed6a801f924595f6e0f3808391198197835abb',
  'StudyComments': nan},
 {'StudyID': 'e4cec40844631c926e2905f514b53643c30cfc9a',
  'StudyDate': '20221030',
  'StudyTime': '000000.000000',
  'StudyDescription': nan,
  'StudyInstanceUID': '1.2.826.0.1.10184103.5.4.175302',
  'StudyStatusID': 'f1ed6a801f924595f6e0f3808391198197835abb',
  'StudyComments': nan},
 {'StudyID': 'e4cec40844631c926e2905f514b53643c30cfc9a',
  'StudyDate': '20221030',
  'StudyTime': '000000.000000',
  'StudyDescription': nan,
  'StudyInstanceUID': '1.2.826.0.1.10184103.5.4.175302',
  'StudyStatusID': 'f1ed6a801f924595f6e0f3808391198197835abb',
  'StudyComments': nan},
 {'StudyID': 'e4cec40844631c926e2905f514b53643c30cfc9a',
  'StudyDate': '20221030',
  'StudyTime': '000000.000000',
  'StudyDescription'

In [272]:
#studies for studies in result_df['studies']

[salty_hash(str(study['StudyID'])) for study in studies for studies in result_df['studies']]


['...']